## Regularization: application with medical microbiology diagnosis

We want to predict the IBD (Inflammatory Bowel Disease) clinical status of patients **given the abundance of microbial species** living in their gut.

Each patient is assigned to a clinical status, and the abundances of known species in their gut are reported in a matrix of size `patient x species`.

We focus here on the seminal metagenomic study by Nielsen H.B. et al, published in 2014.



In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score

# Viz
import matplotlib.pyplot as plt

In [ ]:
# Load the data
ab_data_pd = pd.read_csv("gut_abundances.tsv", sep="\t", header = 0)
descriptor_names = ab_data_pd.columns
ab_data = ab_data_pd.to_numpy()
status_txt =  pd.read_csv("ibd_status.lst", sep="\t", header = None).to_numpy()
ab_data_pd

Take a look at the data: number of samples ($n$), number of descriptors ($p$), names of descriptors (in ```descriptor_names```), etc.

In [ ]:
n,p =#TO COMPLETE (1 expression)
print("Data shape:",n,p)

We now transform the target variable to a binary (0 ==  control, 1 == IBD) variable ```status```:

In [ ]:
status = np.ravel([int(s == "IBD") for s in status_txt])

What is the name of the most correlated species (we denote by S) to the clinical status? Use the ```np.corrcoef``` function to compute the correlation between the data and the status.
You can compute the correlation between the data and the status using ```corr_vector = np.corrcoef(ab_data.transpose(),status))[:-1,p]```, which gives the correlation for each species.

In [ ]:
#TO COMPLETE
 

Create a (naïve) predictor that takes as input the abundance of S and output the clinical status of the patient. Either compute the prediction accuracy or its AUC ROC for every threshold, this will be our baseline predictor to improve on.

In [ ]:
# Split data into 50% train and 50% test subsets
x_train, x_test, y_train, y_test = train_test_split(
    ab_data, status, test_size=0.2, shuffle=True
)
print(x_train.shape)
print(ab_data.shape)

# We remove species not abundant in the train set
col_to_rm = np.where(np.sum(x_train,axis=0) == 0)
x_train = np.delete(x_train,col_to_rm,axis=1)
x_test =  np.delete(x_test,col_to_rm,axis=1)
n,p = x_train.shape

# Index of the best correlated species to patient's status
best_cor_sp =#TO COMPLETE (1 expression)
print(best_cor_sp)

# Compute accuracy 
accuracy = list()
ths = np.linspace(0,int(np.max(x_train[:,best_cor_sp]))+1,1000) # threshold list
for t in ths:
    predictions = (x_train[:,best_cor_sp]>t)
    accuracy += [sum( predictions ==#TO COMPLETE (1 expression)

    
# Plot the decision threshold vs. performance of the predictor

plt.scatter(ths,accuracy)
plt.show()



# "Real" accuracy on test set
# Decision threshold corresponding to the best accuracy.
best_ths =#TO COMPLETE (1 expression)
# Report the accuracy on the test set
#TO COMPLETE
#TO COMPLETE


roc = roc_auc_score(y_test==1,#TO COMPLETE (1 expression)
print("AUC ROC on test set:",roc)


Now we hope that we can do better when using more than 1 descriptor.

We will first use a standard [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Use the [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html ) function of Sklearn to evaluate your predictor in a cross-validation way.

Take a look at what metrics you can find in the attributes of the cross-validation object `cv` and print the average accuracy.

Do you gain in terms of accuracy compared to a single predictor? Check if you are overfitting the data.

In [ ]:
logreg = linear_model.LogisticRegression(solver="liblinear")
cv =#TO COMPLETE (1 expression)


test_score =#TO COMPLETE (1 expression)
print("Average Cross-Validation accuracy on test:",test_score)


# Split data into 50% train and 50% test subsets
x_train, x_test, y_train, y_test = train_test_split(
    ab_data, status, test_size=0.5, shuffle=True
)

# Print example accuracy on 1 fold
#TO COMPLETE
#TO COMPLETE
#TO COMPLETE
 

Check the `coef_` attribute of your model. How many species are you using for taking the decision?

In [ ]:
used_descriptors =#TO COMPLETE (1 expression)
print("The model uses ",used_descriptors,"descriptors")

Considering that only few bacterial species may be responsible for IBD, choose a relevant regularization (see the available penalties for logistic regression [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).



In [ ]:
# Split data into 50% train and 50% test subsets
x_train, x_test, y_train, y_test = train_test_split(
    ab_data, status, test_size=0.5, shuffle=True
)

# Create a regularized model
logreg_regul =#TO COMPLETE (1 expression)
logreg_regul.fit(x_train,y_train)
pred = logreg_regul.predict(x_test)

# Report accuracy on test set
#TO COMPLETE
 

Have you increased the accuracy? Looking at `logreg_regul.coef_`, how many descriptors are you using after regularization?
All things considered, is it a better model?

In [ ]:
used_descriptors =#TO COMPLETE (1 expression)
print("We use ",used_descriptors,"descriptors")

Remember that equivalent accuracy, but with fewer descriptors => more robust and explainable => better model !

Find the optimal regularization strength, by comparing performances on test set and train set. If you have more time, best is to do it by plotting the mean performance in cross-validation, with confidence enveloppe (+/- std deviation) over the folds.


In [ ]:
# Split data into 50% train and 50% test subsets
x_train, x_test, y_train, y_test = train_test_split(
    ab_data, status, test_size=0.5, shuffle=True
)

acc_test = list()
acc_train = list()


reg_strengths = [0.001,0.005,0.01,0.03,0.06,0.1,0.15,0.2,0.3,1]
for c in reg_strengths:
    logreg = linear_model.LogisticRegression(#TO COMPLETE (1 expression)
    cv =#TO COMPLETE (1 expression)
    acc_test +=#TO COMPLETE (1 expression)
    acc_train +=#TO COMPLETE (1 expression)
    
plt.clf()
plt.scatter(reg_strengths,acc_test, label='acc test')
plt.scatter(reg_strengths,acc_train, label='acc train')
plt.legend()
plt.show()

Train a classifier on the full dataset with the optimial regularization strength, and interpret the coefficients. You can check in particular if it is consistent with [this paper](https://pubmed.ncbi.nlm.nih.gov/27999802/) and [this one](https://pubmed.ncbi.nlm.nih.gov/20648002/).

In [ ]:
# c is the optimal penalty given the previous graph (hard-code this number)
c =#TO COMPLETE (1 expression)
logreg = linear_model.LogisticRegression(#TO COMPLETE (1 expression)
logreg.fit(ab_data,status)

# positive association IBD, you can make use of numpy argsort function
order =#TO COMPLETE (1 expression)
order = order.tolist()
print(np.array(descriptor_names)[order[0][0:3]])


# negative association with IBD
order =#TO COMPLETE (1 expression)
print(np.array(descriptor_names)[order[0][0:3]])



## Optional: other classifiers

Use other models for supervised classification (decision trees, SVM, neural nets, etc.) with the IBD data. Evaluate properly the performances, and pay attention to regularization!

